In [ ]:

# below: Source: https://spaceflight.nasa.gov/shuttle/reference/shutref/orbiter/eps/pwrplants.html
"""The voltage and current range of each is 2 kilowatts at 32.5 volts dc, 61.5 amps, to
12 kilowatts at 27.5 volts dc, 436 amps. Each fuel cell is capable of supplying 12 kilowatts peak and 7 kilowatts
maximum continuous power. The three fuel cells are capable of a maximum continuous output of 21,000 watts with
15-minute peaks of 36,000 watts. The average power consumption of the orbiter is expected to be approximately 14,000 watts, or 14 kilowatts, leaving 7 kilowatts average available for payloads.
Each fuel cell will be serviced between flights and reused until each accumulates 2,000 hours of on-line service. """

g_force = 1.0
print 'constant g-force (acceleration) of ' + str(g_force) + 'g'
c = 3.0e+8 # speed of light, m/s
sec_in_1hr = 3600.
m_in_1AU = 1.496e+11
m_in_1pc =  3.0857e+16
amu = 1.66e-27 # atomic mass unit, in kg
fuel_mass_used_in_1reaction_kg = 4.0*amu # pp1: 4 1H nuclei used

acc = g_force*9.81 # m/s^2

spaceship_mass_kg = 1.0e+8 # kg
spaceship_mass_t = spaceship_mass_kg/1000. # kg
v_final = 0.01*c

f_net = spaceship_mass_kg*acc
t_acc_s = v_final/acc
t_acc_hr = t_acc_s/sec_in_1hr

print 'Spaceship mass =  '+str(spaceship_mass_t)+' tonnes'
print 'Net force =  '+str(f_net)+' N'
print 'Time to accelerate to a speed of '+str(v_final/c)+'c =  '+str(t_acc_hr)+' hours'

Qpp1_MeV = 26.732 # assume this (pp1 energy release in MeV) produces the energy
Qpp1_J = Qpp1_MeV*1.6e-13

d_acc_m = 0.5*acc*(t_acc_s)**2
d_acc_AU = d_acc_m/m_in_1AU
print 'Distance travelled during acceleration =  '+str(d_acc_AU)+' AU'
work_done = f_net*d_acc_m
print 'Work done over this distance =  '+str(work_done)+' J'

number_of_reactions_acc = work_done/Qpp1_J
fuel_needed_acc_kg = number_of_reactions_acc*fuel_mass_used_in_1reaction_kg
print 'Fuel mass required for initial acceleration =  '+str(fuel_needed_acc_kg/1000.)+'  tonnes'
print '\n'

space_shuttle_empty_mass_kg = 68585.
space_shuttle_useful_load_kg = 25060.
#space_shuttle_max_takeoff_mass_kg = 108864.

power_per_SS_electric_PP_W = 7000.
number_SS_electric_PP_W = 3
power_factor_increase = 2.0
print 'Multiplying SS total electrical power by a factor of ',power_factor_increase

# Assume same density of spaceship and shuttle (i.e., mass = constant x volume)
volume_ratio = spaceship_mass_kg/(space_shuttle_empty_mass_kg + space_shuttle_useful_load_kg)
print 'Spaceship-space shuttle volume (mass) ratio:  '+str(volume_ratio)

star_distances_pc = {
    'Epsilon Eridani' : 3.212,
    'Proxima Centauri' : 1.301
}
key = 'Epsilon Eridani'

distance_to_star_pc = star_distances_pc[key]
distance_to_star_m = distance_to_star_pc*m_in_1pc
print 'Distance to '+ key +' = '+str(distance_to_star_pc)+' pc'

t_travel_s = (2*t_acc_s)+(distance_to_star_m/v_final)
print 'Total travel time to '+key+' (accn.,cruising,decn.) =  '+str(t_travel_s/(sec_in_1hr*24*365.25))+' years'
print '\n'

habitability_energy = number_SS_electric_PP_W*power_factor_increase*power_per_SS_electric_PP_W*volume_ratio*t_travel_s
print 'Energy required to keep spaceship interior habitable =  '+str(habitability_energy)+' J'

number_of_reactions_hab = habitability_energy/Qpp1_J
fuel_needed_hab_kg = number_of_reactions_hab*fuel_mass_used_in_1reaction_kg
print 'Fuel mass required for habitability =  '+str(fuel_needed_hab_kg/1000.)+'  tonnes'

print 'Total 1H-fusion fuel mass required (accn.,cruising,decn.,hab.) = '+str((fuel_needed_hab_kg+2*fuel_needed_acc_kg)/1000.)+'  tonnes'

In [ ]:
# polynomial fit write-out function
def poly_fit_number_gen_write(f,xdata,ydata,poly_degs_arr,filter_str,logg_val):
    output_names = ['    Coefficients (in order of descending polynomial index):','    Sum of residuals:', '    Rank of coefficient matrix:', '    Singular values of coefficient matrix:', '    Cut-off ratio for small singular values of coefficient matrix:']
    print 'List of degrees for polynomial fitting for grid point: ',poly_degs_arr
    f.write('Fitting results for '+str(filter_str)+' filter, with log(g) = '+str(logg_val)+'\n')
    for i in poly_degs_arr:
        f.write('        Results for polynomial fit of degree ' + str(i) + ': \n')
        c = 0
        for j in np.polyfit(xdata,ydata,i,full=True):
            if (len(np.polyfit(xdata,ydata,i,full=True)) == len(output_names)):
                f.write(output_names[c])
                f.write('\n')
                f.write(str(j))
                f.write('\n')
                c += 1
            elif(len(np.polyfit(xdata,ydata,i,full=True)) > len(output_names)):
                print 'Error: Not enough headings for data'
            else:
                print 'Error: Not enough data fields for output'
        f.write('\n')
    f.write('\n')

# exponential function weighted by Teff^d
def single_poly_x_exp_func(xdata,a,b,c,d):
    y = (xdata**d)*(a*(np.exp(b*xdata))) + c
    return y


# logarithmic function weighted by Teff^d
def single_poly_x_log_func(xdata,a,b,c,d):
    y = (xdata**d) * (a*(np.log10(b*xdata))) + c
    return y

# logarithmic function added to Teff^d
def single_poly_plus_log_func(xdata,a,b,c,d):
    y = (xdata**d) + (a*(np.log10(b*xdata))) + c
    return y

fit_types.append('Power law of logTeff, fitted')
fit_types.append('Modified Poissonian of logTeff, fitted')
fit_types.append('Power law + exponential function of logTeff, fitted')
fit_types.append('Modified Poissonian of Teff, fitted')
fit_types.append('Logarithmic function of Teff, fitted')
fit_types.append('Power law * log function of Teff, fitted')
fit_types.append('Power law + log function of Teff, fitted')

#,method='lm'
#try:
#except RuntimeError:
#    print "Error - curve_fit exponential failed"
#log_fit_A_G5zs_logT, covarr_A_G5zs_lf_logT = curve_fit(log_func,log_Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([1.0e-02,1.0e-02,1.0e-02], [1., 2., 10.]))

spx_exp_fit_A_G5zs_logT, covarr_A_G5zs_spx_logT = curve_fit(single_poly_x_exp_func,log_Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-1.0e+07,-10.,-5.,-10.], [-100., 0., 5., 20.]))
spp_exp_fit_A_G5zs_logT, covarr_A_G5zs_spp_logT = curve_fit(single_poly_plus_exp_func,log_Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-1.0e+05,-10.,-5.,-4.0e+03], [0.,0.,5.,10.]))

print 'logTeff^(n) * exponential fit coefficients: '
print spx_exp_fit_A_G5zs_logT
print 'Covariance matrix: '
print covarr_A_G5zs_spx_logT

print 'logTeff^(n) + exponential fit coefficients: '
print spp_exp_fit_A_G5zs_logT
print 'Covariance matrix: '
print covarr_A_G5zs_spp_logT


#,method='lm'
#try:
spx_exp_fit_A_G5zs, covarr_A_G5zs_spx = curve_fit(single_poly_x_exp_func,Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-100.,-1.0e-03,-10.,-1.], [0., 1.0e-03, 10., 1.]))
spp_exp_fit_A_G5zs, covarr_A_G5zs_spp = curve_fit(single_poly_plus_exp_func,Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-10.,-1.0e-03,-1.,0.], [0., 0., 0., 1.0e-01]))

spx_log_fit_A_G5zs, covarr_A_G5zs_spxl = curve_fit(single_poly_x_log_func,Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-1.2e+02,1.0e-03,-1.5,-2.], [-40., 1.0e+06, 1.5, 1.]))
spp_log_fit_A_G5zs, covarr_A_G5zs_sppl = curve_fit(single_poly_plus_log_func,Teff_5_zs,A_G_5_zs, p0=None, sigma=None,bounds=([-10.,1.0e-07,-1.5,-10.], [10., 20., 5., 1.]))
ax.plot(Teff_5_zs,single_poly_x_log_func(Teff_5_zs,*spx_log_fit_A_G5zs),'b',label=fit_types[8])
ax.plot(Teff_5_zs,single_poly_plus_log_func(Teff_5_zs,*spp_log_fit_A_G5zs),'r',label=fit_types[9])

ax.plot(log_Teff_5_zs,single_poly_x_exp_func(log_Teff_5_zs,*spx_exp_fit_A_G5zs_logT),'b',label=fit_types[1])
ax.plot(log_Teff_5_zs,single_poly_plus_exp_func(log_Teff_5_zs,*spp_exp_fit_A_G5zs_logT),'r',label=fit_types[2])

#ax.plot(Teff_5_zs,single_poly_x_exp_func(Teff_5_zs,-9.26543212e-01,-2.39459351e-04 ,1.02037796e+00,1.4e-03),'g',label='dfgiugdfgdr')

#print 'Testing log curve'
#print Teff_5_zs
#print log_func(Teff_5_zs,0.22744533,0.17575513,0.17663504)

#except RuntimeError:
#    print("Error - curve_fit exponential failed")


"""
#ax.plot(Teff_5_zs,A_G_5_zs,'r',label='Z = Z$_{\odot}$')
#ax.plot(Teff_5_z2,A_G_5_z2,'g',label='Z = 10$^{-2}$ Z$_{\odot}$')

print len(A_X_zs_Tfix)

fig,ax = plt.subplots()
col_map = plt.cm.gnuplot
dict_2D_plot(A_X_zs_Tfix,ax,2,3,var_names,col_map)
plt.show()
"""
ax.plot(Teff_5_zs,single_poly_x_exp_func(Teff_5_zs,*spx_exp_fit_A_G5zs),'b',label=fit_types[5])
ax.plot(Teff_5_zs,single_poly_plus_exp_func(Teff_5_zs,*spp_exp_fit_A_G5zs),'r',label=fit_types[6])
#ax.plot(Teff_5_zs,single_poly_x_exp_func(Teff_5_zs,-9.26543212e-01,-2.39459351e-04 ,1.02037796e+00,1.4e-03),'g',label='dfgiugdfgdr')

#print 'Testing log curve'
#print Teff_5_zs
#print log_func(Teff_5_zs,0.22744533,0.17575513,0.17663504)

"""
#ax.plot(Teff_5_zs,A_G_5_zs,'r',label='Z = Z$_{\odot}$')
#ax.plot(Teff_5_z2,A_G_5_z2,'g',label='Z = 10$^{-2}$ Z$_{\odot}$')
exp_test_begin =  exp_func(Teff_5_zs[0],-7.26540361e-01,-2.39458489e-04,1.02037808e+00)
#log_test_begin =  log_func(Teff_5_zs[0],-7.26540361e-01,-2.39458489e-04,1.02037808e+00)
spx_test_begin = single_poly_x_exp_func(Teff_5_zs[0],9.05023884e+00,-2.01320841e-07,-1.00000000e+01,2.13054689e-02)
spp_test_begin = single_poly_plus_exp_func(Teff_5_zs[0],-7.26149370e-01,-3.33296157e-04,-2.79215137e-01,2.52357409e-02)
print 'Predicted values: ',exp_test_begin,spx_test_begin,spp_test_begin
print 'Actual data value = ',A_G_5_zs[0]
"""
"""
# write out
with open ("gaia_spectra/Teff_poly_fit_logg=5_zs_numbers",'w') as f:
    print '\n    Writing log(g)=5.0, Z = Zsolar model'
    # polynomial Teff fits
    poly_degs_arr = [4,5,6,7]
    poly_fit_number_gen_write(f,Teff_5_zs,A_G_5_zs,poly_degs_arr,'G',5.0)
    poly_fit_number_gen_write(f,Teff_5_zs,A_Gbp_5_zs,poly_degs_arr,'G_bp',5.0)
    poly_fit_number_gen_write(f,Teff_5_zs,A_Grp_5_zs,poly_degs_arr,'G_bp',5.0)
    f.close()
    print 'writing complete'"""

#fit0_p3 = np.polyfit(Teff_5,A_G_5,3)
#print fit0_p3

#fit_G5zs_p3 = np.polyfit(Teff_5_zs,A_G_5_zs,3)
#print fit_G5zs_p3
#fit_G5zs_p2,covout_G_zs = np.polyfit(Teff_5_zs,A_G_5_zs,2,cov=True)
#print fit_G5zs_p2
#print covout_G_zs
fit_G5zs_p4, cov_G5zs_p4 = np.polyfit(Teff_5_zs,A_G_5_zs,4,cov=True)
fit_G5zs_p5, cov_G5zs_p5 = np.polyfit(Teff_5_zs,A_G_5_zs,5,cov=True)
fit_G5zs_p6, cov_G5zs_p6 = np.polyfit(Teff_5_zs,A_G_5_zs,6,cov=True)
fit_G5zs_p7, cov_G5zs_p7 = np.polyfit(Teff_5_zs,A_G_5_zs,7,cov=True)
#print fit_G5zs_p4

#fit_Gbp5zs_p3 = np.polyfit(Teff_5_zs,A_Gbp_5_zs,3)
#print fit_G5zs_p3
#fit_Gbp5zs_p2,covout_Gbp_zs = np.polyfit(Teff_5_zs,A_Gbp_5_zs,2,cov=True)
#print fit_Gbp5zs_p2
#print covout_Gbp_zs
fit_Gbp5zs_p4, cov_Gbp5zs_p4 = np.polyfit(Teff_5_zs,A_Gbp_5_zs,4,cov=True)
fit_Gbp5zs_p5, cov_Gbp5zs_p5 = np.polyfit(Teff_5_zs,A_Gbp_5_zs,5,cov=True)
fit_Gbp5zs_p6, cov_Gbp5zs_p6 = np.polyfit(Teff_5_zs,A_Gbp_5_zs,6,cov=True)
fit_Gbp5zs_p7, cov_Gbp5zs_p7 = np.polyfit(Teff_5_zs,A_Gbp_5_zs,7,cov=True)
#print fit_Gbp5zs_p4

#fit_Grp5zs_p3 = np.polyfit(Teff_5_zs,A_Grp_5_zs,3)
#print fit_G5zs_p3
#fit_Grp5zs_p2,covout_Grp = np.polyfit(Teff_5_zs,A_Grp_5_zs,2,cov=True)
#print fit_Grp5zs_p2
#print covout_Grp
fit_Grp5zs_p4, cov_Grp5zs_p4 = np.polyfit(Teff_5_zs,A_Grp_5_zs,4,cov=True)
fit_Grp5zs_p5, cov_Grp5zs_p5 = np.polyfit(Teff_5_zs,A_Grp_5_zs,5,cov=True)
fit_Grp5zs_p6, cov_Grp5zs_p6 = np.polyfit(Teff_5_zs,A_Grp_5_zs,6,cov=True)
fit_Grp5zs_p7, cov_Grp5zs_p7 = np.polyfit(Teff_5_zs,A_Grp_5_zs,7,cov=True)
#print fit_G5zs_p4

#print type(type(test_list))
for i in np.polyfit(Teff_5_zs,A_Grp_5_zs,7,full=True):
#print type(np.polyfit(Teff_5_zs,A_Grp_5_zs,7,full=True))
    print type(i)
    print i

    # dictionary of average errors
#'G','G_bp','G_rp'
avg_dict = {}
with open ("gaia_spectra/Teff_AGrp_gen_fit_logg=5.0_zs_numbers.txt",'w') as f:
    print '\n    Writing log(g)=5.0, Z = Zsolar model'
    general_fit_number_gen_write(f,fit_types[0],exp_fit_A_G5zs, covarr_A_G5zs_ef,'G_rp',5.0,avg_dict)
    general_fit_number_gen_write(f,fit_types[1],pow_fit_A_G5zs, covarr_A_G5zs_pow,'G_rp',5.0,avg_dict)
    general_fit_number_gen_write(f,fit_types[2],spp_exp_fit_A_G5zs, covarr_A_G5zs_spp,'G_rp',5.0,avg_dict)

    # write results of comparison of averages
    sorted_avg = sorted(avg_dict.items(), key=operator.itemgetter(1))
    f.write('RANKED MEAN FRACTIONAL ERRORS (best to worst) :   ' + '\n')
    for i in sorted_avg:
        f.write("{: <40}".format(str(i[0])) + 2*'\t' + str(i[1]) + '\n')
    f.close()
    print 'writing complete'